In [2]:
from torch import nn
import torch
import numpy as np
from torch.nn.functional import cross_entropy, softmax
from torch.utils.data import DataLoader
import datetime
import utils

In [7]:
class Seq2Seq(nn.Module):
    # enc_v_dim:输入编码器的文本的样本数（输入序列的词汇表大小）
    # dec_v_dim:输入解码器的文本样本数
    # emb_dim:嵌入层向量维度
    # units:lstm层中隐藏层单元的数量
    # max_pred_len:最大预测长度
    # start_token:开始标记
    # end_token:结束标记
    def __init__(self,enc_v_dim,dec_v_dim,emb_dim,units,max_pred_len,start_token,end_token):
        super(Seq2Seq, self).__init__()
        self.units=units
        self.dec_v_dim=dec_v_dim

        # encoder
        self.enc_embeddings=nn.Embedding(enc_v_dim,emb_dim) #编码器嵌入层
        self.enc_embeddings.weight.data.normal_(0,0.1)      #初始化嵌入层权重
        self.encoder = nn.LSTM(emb_dim,units,1,batch_first=True)    #LSTM层

        # decoder
        self.dec_embeddings = nn.Embedding(dec_v_dim,emb_dim)    #解码器嵌入层
        self.attn=nn.Linear(units,units)
        self.decoder_cell=nn.LSTMCell(emb_dim,units)
        self.decoder_dense = nn.Linear(units*2,dec_v_dim)

        self.opt=torch.optim.Adam(self.parameters(),lr=0.001)   #Adam优化器
        self.max_pred_len=max_pred_len  #最大预测长度
        self.start_token=start_token    #开始标记
        self.end_token=end_token        #结束标记

    # 编码，这个函数作为带训练文本输入的入口
    def encode(self,x):
        embedded = self.enc_embeddings(x)   #将x作为参数传入编码器的嵌入层，x对应的输出嵌入向量，emb.shape=[n,step,emb](样本数，时间步，嵌入向量维度)
        hidden = (torch.zeros(1,x.shape[0],self.units),torch.zeros(1,x.shape[0],self.units))    #LSTM的初始状态，h表示lstm的初始隐藏状态，c表示lstm的初始细胞状态([1,n,units],[1,n,units)
        o,(h,c)=self.encoder(embedded,hidden)   #将嵌入向量和初始状态喂入lstm
        return o,h,c    #返回输出数据和新的隐藏状态和细胞状态

    # 基于注意力机制的解码过程
    def inference(self,x,return_align=False):
        self.eval() #开启评估模式
        o,hx,cx=self.encode(x)  #将x传入编码器，返回输出o和新的隐藏状态hx以及新的初始化状态cx，维度[n, step, units], [num_layers * num_directions, n, units] * 2
        """
        对于单层单向的 LSTM，`hx`或`cx`的第一维度是1，表示只有一个层。`num_layers`表示 LSTM 层的数量，对于单层 LSTM，`num_layers`为1。而`num_directions`表示 LSTM 的方向数，有两种情况：
            - 单向 LSTM：`num_directions`为1，表示只有一个方向。
            - 双向 LSTM：`num_directions`为2，表示正向和反向两个方向。
        在双向 LSTM 中，隐藏状态和细胞状态的第一维度会乘以`num_directions`，以保存每个方向的状态。所以如果是多层的双向 LSTM，`hx`和`cx`的第一维度可以表示为`num_layers * num_directions`。
        """

        #创建开始标记
        start = torch.ones(x.shape[0],1)    # [n, 1]
        start[:,0] = torch.tensor(self.start_token)
        start= start.type(torch.LongTensor)
        # 将开始标记转换成嵌入向量作为解码器的输入
        dec_emb_in = self.dec_embeddings(start) # [n, 1, emb_dim]
        dec_emb_in = dec_emb_in.permute(1,0,2)  # [1, n, emb_dim]
        dec_in = dec_emb_in[0]                  # [n, emb_dim]

        output = []
        for i in range(self.max_pred_len):
            # 计算注意力权重
            attn_prod = torch.matmul(self.attn(hx.unsqueeze(1)),o.permute(0,2,1)) # 将编码器的隐藏状态hx在维度1上扩展，与解码器的输出o进行点积运算，得到注意力分数attn_prod[n, 1, step]
            att_weight = softmax(attn_prod, dim=2)  # 得到注意力权重att_weight[n, 1, step]

            # 计算上下文向量context
            # 上下文向量（Context Vector）是在注意力机制中用于表示编码器输出在解码器中的加权汇总信息。它起到将编码器的输出与当前解码器的隐藏状态相结合的作用，以便更好地捕捉相关信息。
            context = torch.matmul(att_weight,o)    # [n, 1, units]
            # attn_prod = torch.matmul(self.attn(o),hx.unsqueeze(2))  # [n, step, 1]
            # attn_weight = softmax(attn_prod,dim=1)                  # [n, step, 1]
            # context = torch.matmul(o.permute(0,2,1),attn_weight)    # [n, units, 1]

            # 更新隐藏状态(hx, cx)
            hx, cx = self.decoder_cell(dec_in, (hx, cx))
            # 将上下文向量context和更新后的隐藏状态hx在维度1上进行拼接
            hc = torch.cat([context.squeeze(1),hx],dim=1)           # [n, units *2]
            # hc = torch.cat([context.squeeze(2),hx],dim=1)           # [n, units *2]

            # 输出结果
            result = self.decoder_dense(hc) #将特征向量hc通过线性层decoder_dense映射为输出空间
            result = result.argmax(dim=1).view(-1,1)    #取得概率最大的输出值作为当前时间步的输出
            # 将当前时间步的输出作为解码器的嵌入输入dec_in，进入下一个时间步的循环
            dec_in=self.dec_embeddings(result).permute(1,0,2)[0]
            output.append(result)
        output = torch.stack(output,dim=0)  #将输出结果堆叠到output中
        self.train()    #将模型设为训练模式

    # 实现了训练时的模型前向传播过程，包括了编码器的处理、注意力计算、解码器的处理和输出
    def train_logit(self,x,y):
        o,hx,cx = self.encode(x)    # 将x输入编码器，返回编码器输出o和隐藏状态hx/cx[n, step, units], [num_layers * num_directions, n, units] * 2
        hx,cx = hx[0],cx[0]         # 获取编码器的最终隐藏状态[n, units]
        dec_in = y[:,:-1]           # 从目标序列y中移除最后一个时间步，得到解码器的输入序列[n, step]
        dec_emb_in = self.dec_embeddings(dec_in)    # [n, step, emb_dim]
        dec_emb_in = dec_emb_in.permute(1,0,2)      # [step, n, emb_dim]
        output = []
        for i in range(dec_emb_in.shape[0]):
            # General Attention:
            # score(ht,hs) = (ht^T)(Wa)hs
            # hs is the output from encoder
            # ht is the previous hidden state from decoder
            # self.attn(o): [n, step, units]
            attn_prod = torch.matmul(self.attn(hx.unsqueeze(1)),o.permute(0,2,1)) # 计算注意力分数[n, 1, step]
            att_weight = softmax(attn_prod, dim=2)  # 得到注意力权重[n, 1, step]
            context = torch.matmul(att_weight,o)    # 计算上下文向量[n, 1, units]
            # attn_prod = torch.matmul(self.attn(o),hx.unsqueeze(2))  # [n, step, 1]
            # attn_weight = softmax(attn_prod,dim=1)                  # [n, step, 1]
            # context = torch.matmul(o.permute(0,2,1),attn_weight)    # [n, units, 1]
            hx, cx = self.decoder_cell(dec_emb_in[i], (hx, cx))     # [n, units]
            hc = torch.cat([context.squeeze(1),hx],dim=1)           # [n, units *2]
            # hc = torch.cat([context.squeeze(2),hx],dim=1)           # [n, units *2]
            result = self.decoder_dense(hc)                              # [n, dec_v_dim]
            output.append(result)
        output = torch.stack(output,dim=0)  # [step, n, dec_v_dim]
        return output.permute(1,0,2)        # [n, step, dec_v_dim]

    # 完成了一次训练的过程，包括了前向传播、损失计算、反向传播和参数更新
    def step(self,x,y):
        self.opt.zero_grad()    #将梯度清零（以免以前的梯度干扰后续的代码执行）
        batch_size = x.shape[0] #批次大小
        logit = self.train_logit(x,y)   #前向传播，输出预测值
        dec_out = y[:,1:]   #从目标序列 y 中移除第一个时间步，得到解码器的输出序列
        loss = cross_entropy(logit.reshape(-1,self.dec_v_dim),dec_out.reshape(-1))#计算交叉熵损失，将模型输出和目标序列进行比较并计算损失值
        loss.backward() #反向传播，计算模型参数的梯度
        self.opt.step() #根据优化器的更新规则，更新模型参数
        return loss.detach().numpy()    #返回损失值的 numpy 数组形式


def train():
    dataset = utils.DateData(4000)
    print("Chinese time order: yy/mm/dd ",dataset.date_cn[:3],"\nEnglish time order: dd/M/yyyy", dataset.date_en[:3])
    print("Vocabularies: ", dataset.vocab)
    print(f"x index sample:  \n{dataset.idx2str(dataset.x[0])}\n{dataset.x[0]}",
    f"\ny index sample:  \n{dataset.idx2str(dataset.y[0])}\n{dataset.y[0]}")
    loader = DataLoader(dataset,batch_size=32,shuffle=True)
    model = Seq2Seq(dataset.num_word,dataset.num_word,emb_dim=16,units=32,max_pred_len=11,start_token=dataset.start_token,end_token=dataset.end_token)
    for i in range(100):
        for batch_idx , batch in enumerate(loader):
            bx, by, decoder_len = batch
            bx = bx.type(torch.LongTensor)  #将数据格式转换为LongTensor类型
            by = by.type(torch.LongTensor)
            loss = model.step(bx,by)
            if batch_idx % 70 == 0:
                target = dataset.idx2str(by[0, 1:-1].data.numpy())
                pred = model.inference(bx[0:1])
                res = dataset.idx2str(pred[0].data.numpy())
                src = dataset.idx2str(bx[0].data.numpy())
                print(
                    "Epoch: ",i,
                    "| t: ", batch_idx,
                    "| loss: %.3f" % loss,
                    "| input: ", src,
                    "| target: ", target,
                    "| inference: ", res,
                )

if __name__ == '__main__':
    train()


Chinese time order: yy/mm/dd  ['31-04-26', '04-07-18', '33-06-06'] 
English time order: dd/M/yyyy ['26/Apr/2031', '18/Jul/2004', '06/Jun/2033']
Vocabularies:  {'Jul', 'Feb', '4', 'Dec', '<GO>', '7', 'Oct', '2', 'Aug', 'Mar', '<EOS>', '1', '/', '3', 'Nov', 'Apr', 'Jan', '8', '0', 'Jun', '<PAD>', '-', '9', '6', 'May', '5', 'Sep'}
x index sample:  
31-04-26
[6 4 1 3 7 1 5 9] 
y index sample:  
<GO>26/Apr/2031<EOS>
[14  5  9  2 15  2  5  3  6  4 13]


RuntimeError: hidden0 has inconsistent hidden_size: got 1, expected 32

# 细胞状态
在循环神经网络（RNN）中，细胞状态（cell state）是一种记忆机制，用于在不同时间步之间传递和存储信息。

在长短期记忆网络（LSTM）中，细胞状态是 LSTM 的核心组成部分。它可以看作是 LSTM 网络的长期记忆，用于捕捉输入序列中的相关信息并将其传递给后续时间步。细胞状态在 LSTM 中通过门控机制进行更新和调节，从而控制信息的流动和遗忘。

细胞状态在 LSTM 中的更新是通过遗忘门（forget gate）、输入门（input gate）和输出门（output gate）来实现的。遗忘门决定是否保留之前的细胞状态信息，输入门决定更新的新信息，输出门决定将哪些信息传递给下一个时间步。

细胞状态的设计使得 LSTM 能够在处理长序列数据时更好地捕捉长期依赖关系，并且在训练过程中可以通过反向传播进行梯度更新。通过使用细胞状态，LSTM 能够有效地处理序列数据，例如自然语言处理、语音识别等任务。